<a href="https://colab.research.google.com/github/saminhozs/StatisticalFinance/blob/main/Risk_Portfolios_Comparaison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import time
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

pd.options.plotting.backend = 'plotly'

In [3]:
prices = pd.read_excel("20210831 - SMI.xlsx", sheet_name='RI M', index_col=0, parse_dates=True)
returns = prices.pct_change()
revenue = pd.read_excel("20210831 - SMI.xlsx", sheet_name='REV', index_col=0, parse_dates=True).fillna(method='ffill') 
empl = pd.read_excel("20210831 - SMI.xlsx", sheet_name='EMPL', index_col=0, parse_dates=True).fillna(method='ffill') 
market_cap = pd.read_excel("20210831 - SMI.xlsx", sheet_name='MV M', index_col=0, parse_dates=True) 
PE_ratio = pd.read_excel("20210831 - SMI.xlsx", sheet_name='PE', index_col=0, parse_dates=True) 
SMI_index = pd.read_excel("20210831 - SMI.xlsx", sheet_name='OTHER M', index_col=0, parse_dates=True)["SWISS MARKET (SMI), total return"]
rf = pd.read_excel("20210831 - SMI.xlsx", sheet_name='OTHER M', index_col=0, parse_dates=True)["IBA CHF IBK. LIBOR 1M"]/1200

In [4]:
#Equally Weighted

wEW = pd.DataFrame(1, index = market_cap.index, columns = market_cap.columns)
wEW = wEW.div(market_cap.notna().sum(axis=1), axis=0)
wEW[market_cap.isnull()] = 0

In [5]:
fig_weights = go.Figure()
fig_weights.add_trace(go.Bar(x=wEW.columns, y=wEW.iloc[-1], name="Equal weights"))
fig_weights.show()

In [6]:
equalIndexReturn = wEW.mul(returns).sum(axis=1)
equalIndex = (1+equalIndexReturn).cumprod()

In [7]:
SMIIndex = SMI_index/SMI_index.iloc[0]

index_df = pd.Series.to_frame(SMIIndex, name="Official SMI")
index_df["Equal Weighted"] = equalIndex

In [8]:
# Minimum Variance
wMinVar = pd.DataFrame(index = returns.index, columns = returns.columns)
window_covariance_matrix = 36
num_periods = wMinVar.shape[0]

lb = 0.0
ub = 0.3

startTime = time.time()

for row in range(window_covariance_matrix+1, num_periods):
  retour = returns.iloc[row - window_covariance_matrix:row,]
  valid = retour.notna().all().to_list()
  retour_noNan = retour.dropna(axis = 1)

  wMV = np.zeros(retour.shape[1])

  num_assets = retour_noNan.shape[1]
  sigma = retour_noNan.cov()
  e = np.ones((num_assets, 1))
  x0 = np.ones((num_assets, 1)) / num_assets

  objfunction_MV = lambda x: x.T @ sigma @ x
  Aeq = e.T
  beq = 1
  equation_cons1 = lambda x: np.dot(Aeq, x) - beq
  cons_1 = ({'type': 'eq', 'fun': equation_cons1})

  result = minimize(objfunction_MV, x0, method='SLSQP', tol=1e-8, bounds = [(lb,ub)]*num_assets, constraints=cons_1)

  wMV[valid] = result.x
  wMinVar.iloc[row] = wMV

print(f"Time to execute: {time.time() - startTime:.0f} seconds")

Time to execute: 68 seconds


In [9]:
fig_weights.add_trace(go.Bar(x = wEW.columns, y = wMinVar.iloc[-1], name = 'MinVar Weights'))
fig_weights.show()

In [10]:
minvarIndexreturn = wMinVar.mul(returns).sum(axis = 1)
minvarIndex = (1+minvarIndexreturn).cumprod()

index_df['Min Variance'] = minvarIndex

In [11]:
#Maximum Diversification
wMaxDiv = pd.DataFrame(index = market_cap.index, columns = market_cap.columns).fillna(0)
window_covariance_matrix = 36
num_periods = wMaxDiv.shape[0]

lb = 0.0
ub = 0.3

startTime = time.time()

for row in range(window_covariance_matrix +1, num_periods):
  retour = returns.iloc[row-window_covariance_matrix:row,]
  valid = retour.notna().all().to_list()
  retour_noNan = retour.dropna(axis = 1)
  
  wMD = np.zeros(wMaxDiv.shape[1])

  num_assets = retour_noNan.shape[1]
  sigma = retour_noNan.cov()
  sigmaVec = np.diag(sigma)
  e = np.ones((num_assets, 1))
  x0 = np.ones((num_assets, 1)) / num_assets


  objfunctionMD = lambda x: (-1*x.T @ sigmaVec) / ((x.T @ sigma @ x) ** 0.5)
  
  Aeq = e.T
  beq = 1
  equation_cons1 = lambda x: (Aeq @ x) - beq
  cons_1 = ({'type':'eq', 'fun':equation_cons1})

  result = minimize(objfunctionMD, x0, method='SLSQP', tol=1e-8, bounds=[(lb, ub)]*num_assets, constraints=cons_1)

  wMD[valid] = result.x
  wMaxDiv.iloc[row] = wMD

print(f"Time to execute: {time.time() - startTime:.0f} seconds")

Time to execute: 34 seconds


In [12]:
fig_weights.add_trace(go.Bar(x = wEW.columns, y = wMaxDiv.iloc[-1], name = 'Max Diversification'))
fig_weights.show()


In [13]:
maxdivIndexReturn = wMaxDiv.mul(returns).sum(axis = 1)
maxdivIndex = (1 + maxdivIndexReturn).cumprod()

index_df['Max Diversification'] = maxdivIndex

In [14]:
#Equal Risk Contribution
def ERCfunc(w, sigma):
    x = 0
    R = np.dot(sigma, w)
    for i in range(len(w)):
        for j in range(len(w)):
            x = x + (w[i]*R[i] - w[j]*R[j])**2
    return x

In [15]:
wERC = pd.DataFrame(index = market_cap.index, columns = market_cap.columns)
window_covariance_matrix = 36
num_periods = wERC.shape[0]

lb = 0.0
ub = 0.3

startTime = time.time()

for row in range(window_covariance_matrix+1, num_periods):
  retour = returns.iloc[row-window_covariance_matrix:row,]
  valid = retour.notna().all().to_list()
  retour_noNan = retour.dropna(axis = 1)

  wERCnp = np.zeros(retour.shape[1])

  num_assets = retour_noNan.shape[1]
  sigma =  retour_noNan.cov()
  e = np.ones((num_assets, 1))
  x0 = np.ones((num_assets, 1)) / num_assets

  objfunctionERC = lambda x: ERCfunc(x, sigma)

  Aeq = e.T
  beq = 1
  equation_cons1 = lambda x: (Aeq @ x) - beq
  cons_1 = ({'type':'eq', 'fun':equation_cons1})

  result = minimize(objfunctionERC, x0, method = 'SLSQP', tol=1e-8, bounds = [(lb,ub)]*num_assets, constraints = cons_1)

  wERCnp[valid] = result.x
  wERC.iloc[row] = wERCnp

print(f"Time to execute: {time.time() - startTime:.0f} seconds")

Time to execute: 14 seconds


In [16]:
fig_weights.add_trace(go.Bar(x = wEW.columns, y = wERC.iloc[-1], name = 'Equal Risk Contribution'))
fig_weights.show()

In [17]:
ercIndexreturn = wERC.mul(returns).sum(axis = 1)
ercIndex = (1+ercIndexreturn).cumprod()

index_df['Equal Risk Contribution'] = ercIndex

In [18]:
fig_graph = go.Figure()
fig_graph.add_trace(go.Scatter(x = index_df.index, y = index_df['Official SMI'], name = 'Official SMI'))
fig_graph.add_trace(go.Scatter(x = index_df.index, y = index_df['Equal Weighted'], name = 'EW'))
fig_graph.add_trace(go.Scatter(x = index_df.index, y = index_df['Min Variance'], name = 'MVP'))
fig_graph.add_trace(go.Scatter(x = index_df.index, y = index_df['Max Diversification'], name = 'MDP'))
fig_graph.add_trace(go.Scatter(x = index_df.index, y = index_df['Equal Risk Contribution'], name = 'ERC'))
fig_graph.show()

In [19]:
index_df_rebased = index_df/index_df.loc["Dec 2002"].values
index_df_rebased = index_df_rebased.loc["Dec 2002":]
rf = rf.loc["Dec 2002":"Aug 2021"]

returns = index_df_rebased.pct_change()

cagr = (index_df_rebased.iloc[-1]/index_df_rebased.iloc[0])**(12/len(index_df_rebased))-1

stats = pd.DataFrame(cagr, columns=["CAGR"]) 
stats['Volatility'] = returns.std()*(12**0.5)
stats['Excess Return'] = returns.sub(rf, axis = 0).mean()*12
stats['Sharpe Ratio'] = stats['Excess Return'] / stats['Volatility']
stats['Tracking Error'] = returns.sub(returns['Official SMI'], axis = 0).std()*(12**0.5)
stats['Information Ratio'] = returns.sub(returns['Official SMI'], axis = 0).mean()*12/stats['Tracking Error']
maxdd = index_df/np.maximum.accumulate(index_df)-1
stats['Maximum Drawdown'] = maxdd.min()
stats['Sterling Ratio'] = stats['Excess Return'] / stats['Maximum Drawdown']

stats.style.format({
    "CAGR": '{:,.2%}'.format,
    "Volatility": '{:,.2%}'.format,
    "Excess Return": '{:,.2%}'.format,
    "Sharpe Ratio": '{:,.2f}'.format,
    "Tracking Error": '{:,.2%}'.format,
    "Information Ratio": '{:,.2f}'.format,
    "Maximum Drawdown": '{:,.2%}'.format,
    "Sterling Ratio": '{:,.2f}'.format
})

,CAGR,Volatility,Excess Return,Sharpe Ratio,Tracking Error,Information Ratio,Maximum Drawdown,Sterling Ratio
Official SMI,8.50%,12.62%,8.89%,0.70,0.00%,nan,-48.47%,-0.18
Equal Weighted,13.23%,15.37%,13.58%,0.88,6.51%,0.72,-52.86%,-0.26
Min Variance,10.44%,10.70%,10.46%,0.98,7.01%,0.22,-33.64%,-0.31
Max Diversification,12.59%,18.14%,13.50%,0.74,11.57%,0.40,-61.31%,-0.22
Equal Risk Contribution,12.44%,14.60%,12.76%,0.87,6.00%,0.64,-52.58%,-0.24
